In [187]:
import numpy as np
import pandas as pd
import pickle
import numbers
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from tkinter import *
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
pd.options.display.max_seq_items = 2000

In [188]:
df = pickle.load(open('houses.p','rb'))

In [189]:
df.shape

(78295, 221)

In [190]:
compare_features = ['bathroom.badkamer','feature.zwembad','rooms','bedrooms','bathroom.aparte toilet','balcony.balkon','feature.sauna']

In [191]:
df[compare_features].isnull().sum()

bathroom.badkamer           0
feature.zwembad             0
rooms                       0
bedrooms                  986
bathroom.aparte toilet      1
balcony.balkon              0
feature.sauna               0
dtype: int64

In [192]:
df['lotsurface'] = df['lotsurface'].fillna(-1)

In [193]:
isinstance('a', numbers.Number)

False

In [194]:
# weights --> dictionary of feature name to importance
# data --> pandas datafram of houses with all relevant features
# rcf --> list of all relevant features 
# puf --> list of all potentially upgradable features -should be a subset of relevant features 
# thres --> maximum distance a house can have from the target house if it is to be put in a reference set 
# target_house

def normalize_column(x):
    min_max_scaler = MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    return x_scaled, min_max_scaler

def weightedL2(a,b,w):
    q = a-b
    
    return np.sqrt((w*q*q).sum())

# rcf --> relevent comparable features rcfv--> relevant comparable feature values puf --> potential upgradable features
def get_reference_indices(weights, data, thres, target_values):
    reference_indices = []
    for i, house in enumerate(data):
#         print(weightedL2(target_values, house,weights))
        if weightedL2(target_values, house,weights) <  thres:
#             print(weightedL2(target_values, house,weights))
            reference_indices.append(i)
    return reference_indices

def get_reference_sets(weights, rcf, ccf, data, target_house, thres=1):
    df_ans = data.copy()
    df = data[rcf+ccf] # We only look only at the columns that were determined to be relevant for comparison
    t_house = target_house[rcf+ccf]
    for col in ccf:
        val = t_house[col].iloc[0]
        df = df[df[col]==val]
    for col in df[rcf].columns:
        if (df[col].dtype in [bool,float,int,np.int64]):
            a, b = normalize_column(df[[col]].values.astype(float))
            df[col] = a
            t_house[col] = b.transform(t_house[[col]].values.astype(float))
    target_values = t_house[rcf].values
    data_array = df[rcf].values
#     return target_values
    weights_arr = np.array(list(weights.copy().values()))
    reference_index = get_reference_indices(weights_arr, data_array, thres, target_values)
    return df_ans.iloc[reference_index]

In [195]:
weights = {"volume" : 0.275818,
            "livingspace" : 0.251524,
            "Gemiddelde woningwaarde:x 1 000 euro" : 0.113488,
#            "housetype" : 0.054045,
           "V1.x":0.036266,
           "lotsurface":0.029131,
           "yearofconstruction":0.020824,
           "longitude":0.017308,
           'latitude':0.012521}
#            'housesubtype':0.011934}

#Normalize weights 
sum_vals = sum(weights.values())
for key in weights.keys():
    weights[key] = weights[key] / sum_vals
    
rcf = list(weights.keys())
#ccf = ["housesubtype","housetype"]
#ccf = ['feature.zwembad','feature.sauna', 'feature.glasvezelkabel']
#       'feature.stoomcabine', 'feature.stoomcabine','feature.jacuzzi']
thresh = 1
t_house = pd.DataFrame(df.iloc[1]).T

In [196]:
df[compare_features].iloc[1]

bathroom.badkamer             1
feature.zwembad           False
rooms                         2
bedrooms                      1
bathroom.aparte toilet        1
balcony.balkon             True
feature.sauna             False
Name: 57584, dtype: object

In [197]:
result_1 = get_reference_sets(weights=weights, rcf = rcf, ccf = compare_features, data= df, target_house= t_house,thres=thresh)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

In [126]:
# User input 
Volume = float(input("Enter House Volume in m3 : "))
Living_space = float (input("Enter living space in m2 : "))
House_type = input("Please enter the type of the house : ")
Postal_code = input('Please enter a 4 digit postal code : ')
Lot_surface = float( input('Please entere the lot surface in m2 : '))
Year_consrt = int( input("Please enter year of construction : "))
Long = float( input("Please enter longtitude : "))
Lang = float( input("Please enter Langtitude : "))
House_sub_type = input("Please enter house sub type : ")

Enter House Volume in m3 : 


ValueError: could not convert string to float: 

In [213]:
def feature_importance (result_1, rcf, compare_features):
    #This function will fit a simple regression in the data and return the feature importances for each upgradable feature
    # It will also return the test R2 score
    #drop NaN values #note we took the easy approach and dropped everything 
    df_result = result_1[rcf + compare_features + ['endprice']].dropna()
    #scaling the data
    scaler = StandardScaler()
    scaler.fit (df_result[rcf + compare_features])
    #creating X,y and spliting the dataset into train and test set
    X = scaler.transform(df_result[ rcf + compare_features])
    y = df_result.endprice
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    #fitting the regression
    reg = LinearRegression().fit(X_train, y_train)
    score = reg.score(X_test, y_test)
    feature_imp = pd.Series(data=reg.coef_, index = rcf+compare_features)
    return (score,feature_imp)

In [223]:
score, importance_df = feature_importance (result_1, rcf, compare_features)
print ("Regression R2 score : ", score,'\n\nfeature importances : \n',importance_df)

Regression R2 score :  0.6709910966621175 

feature importances : 
 volume                                 -1.965550e-03
livingspace                             3.180225e-01
Gemiddelde woningwaarde:x 1 000 euro    7.731035e-02
V1.x                                   -7.781489e-02
lotsurface                              3.604252e-02
yearofconstruction                      5.710460e-02
longitude                               7.377556e-03
latitude                               -3.095533e-02
bathroom.badkamer                       1.063085e-02
feature.zwembad                         4.163336e-17
rooms                                  -6.908340e-02
bedrooms                                4.341670e-02
bathroom.aparte toilet                  6.217529e-03
balcony.balkon                          2.201905e-02
feature.sauna                           0.000000e+00
dtype: float64


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
